In [1]:
%load_ext autoreload
%autoreload 2    

In [2]:
DB_NAME = "./research.db"

In [3]:
# import os
# os.remove(DB_NAME)

In [4]:
from broai.llm_management.ollama import BedrockOllamaChat

bedrock_model = BedrockOllamaChat(model_name='us.meta.llama3-2-11b-instruct-v1:0')

In [5]:
bedrock_model.run(
    system_prompt="You are the best bro, Andy.",
    messages=[
        bedrock_model.UserMessage(text="Hello Andy.")
    ]
)

"Hello! It's great to chat with you. How's your day going so far?"

# Parsing markdown

In [6]:
from broai.experiments.chunk import split_markdown, consolidate_markdown, get_markdown_sections, split_overlap
from broai.interface import Context, Contexts

In [7]:
with open("./docs/storm.md", "r") as f:
    markdown_text = f.read()

In [8]:
chunks = split_markdown(markdown_text)

Markdown headings: max(4)


/tmp/ipykernel_15313/3854812085.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_markdown
  chunks = split_markdown(markdown_text)


In [9]:
consolidated_chunks = consolidate_markdown(chunks)

/tmp/ipykernel_15313/1728333434.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: consolidate_markdown
  consolidated_chunks = consolidate_markdown(chunks)


In [10]:
sections = get_markdown_sections(consolidated_chunks)

/tmp/ipykernel_15313/4124938538.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: get_markdown_sections
  sections = get_markdown_sections(consolidated_chunks)


In [11]:
contexts = Contexts()
source = ".docs/test1/storm.md"
for section, chunk in zip(sections, consolidated_chunks):
    contexts.add_context(Context(context=chunk, metadata={"section": section, "source": source, "type": "document"}))
len(contexts.contexts)

50

In [12]:
new_contexts = split_overlap(contexts.contexts, max_tokens=1000, overlap=300)
new_contexts = Contexts(contexts=new_contexts)
len(new_contexts.contexts)

/tmp/ipykernel_15313/2598309194.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: split_overlap
  new_contexts = split_overlap(contexts.contexts, max_tokens=1000, overlap=300)


63

# Create: raw_vs

In [13]:
from broai.experiments.vector_store import DuckVectorStore
import json
from broai.interface import Context, Contexts

/home/ec2-user/SageMaker/research_ai/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
from broai.experiments.huggingface_embedding import BAAIEmbedding, EmbeddingDimension
baai_em = BAAIEmbedding()

/tmp/ipykernel_15313/2466456318.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: BAAIEmbedding
  baai_em = BAAIEmbedding()
Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 151055.37it/s]


In [15]:
from broai.experiments.cross_encoder import ReRanker
rr = ReRanker()

/tmp/ipykernel_15313/2186289470.py:2: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: ReRanker
  rr = ReRanker()


In [16]:
raw_vs = DuckVectorStore(db_name=DB_NAME, table="raw", embedding=baai_em)

/tmp/ipykernel_15313/1670256487.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  raw_vs = DuckVectorStore(db_name=DB_NAME, table="raw", embedding=baai_em)
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [17]:
raw_vs.add_contexts(new_contexts.contexts)

In [18]:
raw_vs.read_all()

,id,context,metadata,type,embedding,created_at,updated_at
0,30c93386-73ce-4082-83cc-5b4d9bc16201,# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024\n\nAss...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.04901123, -0.010894775, -0.02658081, -0.02...",2025-04-27 00:03:01.646334,2025-04-27 00:03:01.646334
1,17b93796-f7c1-4631-87db-2c6f1b9b8777,Abstract\n\nWe study how to apply large langua...,"{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[0.0079193115, -0.016906738, -0.025924683, 0.0...",2025-04-27 00:03:01.646399,2025-04-27 00:03:01.646399
2,40339971-d18d-4ca4-a43d-017167bf5665,1 Introduction\n\nLarge language models (LLMs)...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.055419922, 0.009933472, -0.023742676, -0.0...",2025-04-27 00:03:01.646423,2025-04-27 00:03:01.646423
3,27714503-d35d-4156-97ba-1258d347a2d9,2 FreshWiki\n\nWe study generating Wikipedia-l...,"{""section"":""2 FreshWiki"",""source"":"".docs/test1...",document,"[-0.0074806213, 0.011520386, -0.0463562, 0.016...",2025-04-27 00:03:01.646443,2025-04-27 00:03:01.646443
4,1ec83a5a-25c1-40ff-9d0a-8ba84debea3d,"<span id=""page-2-0""></span>2.1 The FreshWiki D...","{""section"":""2.1 The FreshWiki Dataset"",""source...",document,"[-0.06866455, -0.033294678, -0.03250122, 0.028...",2025-04-27 00:03:01.646460,2025-04-27 00:03:01.646460
...,...,...,...,...,...,...,...
58,8991673c-5911-486a-b340-b796726e4195,"**### Red Light Fever**\n\nRed Light Fever, re...","{""section"":""**### Red Light Fever**"",""source"":...",document,"[0.007835388, -0.045318604, -0.06188965, 0.032...",2025-04-27 00:03:01.647121,2025-04-27 00:03:01.647121
59,e74c371f-98f5-4fd9-b292-eba665a3d110,"**## Get the Money**\n\nGet the Money, the thi...","{""section"":""**## Get the Money**"",""source"":"".d...",document,"[0.00982666, -0.018432617, -0.045654297, -0.00...",2025-04-27 00:03:01.647137,2025-04-27 00:03:01.647137
60,f43a9814-f436-42ed-a3fe-23b87695879c,**# Collaborations and Guest Appearances**\n\n...,"{""section"":""**# Collaborations and Guest Appea...",document,"[-0.013725281, -0.03591919, -0.05432129, 0.031...",2025-04-27 00:03:01.647152,2025-04-27 00:03:01.647152
61,bbdf145a-79b2-4c2c-bd0b-9197b54e55c3,"**# Tragic Passing**\n\nTaylor Hawkins, the es...","{""section"":""**# Tragic Passing**"",""source"":"".d...",document,"[0.010620117, -0.028808594, -0.038635254, -0.0...",2025-04-27 00:03:01.647183,2025-04-27 00:03:01.647183


In [19]:
query = "What is STORM?"
retrieved_contexts = raw_vs.vector_search(search_query=query, context=True)
reranked_raw, scores = rr.run(query, retrieved_contexts)
for res in reranked_raw:
    print(res.context)
    print("="*20)

8 Conclusion

We propose STORM, an LLM-based writing system that automates the pre-writing stage for creating Wikipedia-like articles from scratch. We curate the FreshWiki dataset and establish evaluation criteria to study the generation of grounded longform articles. Experimental results demonstrate that the question asking mechanism in STORM improves both the outline and article quality. With the improved breadth and depth, STORM helps surface new challenges for grounded writing systems through expert evaluation. The experienced Wikipedia editors in our study unanimously agree that STORM is helpful for their pre-writing stage.

<span id="page-13-2"></span>B Pseudo Code of STORM

In [§3,](#page-2-8) we introduce STORM, a framework that automates the pre-writing stage by discovering different perspectives, simulating information-seeking conversations, and creating a comprehensive outline. Algorithm [1](#page-16-2) displays the skeleton of STORM.

We implement STORM with zero-shot promp

# Create: enrich_vs

In [21]:
from broai.prompt_management.core import PromptGenerator
from broai.prompt_management.interface import Persona, Instructions, Example, Examples

In [22]:
from pydantic import BaseModel, Field

class Summary(BaseModel):
    summary:str = Field(description="a short summary based on the provided context.")

class InputContext(BaseModel):
    context:str = Field(description="the input context.")

In [23]:
enrich_prompt = PromptGenerator(
    persona=Persona(name="Andy", description="The best editor who can summarize anything."),
    instructions=Instructions(
        instructions=[
            "summarize the context in concise manner.",
            "do not make up your summary, it must be based on the context only."
        ],
        cautions=[
            "respond in specified JSON format as in examples."
        ],
    ),
    structured_output=Summary,
    examples=Examples(examples=[
        Example(setting="Academic journal", input=InputContext(context="This is the input context."), output=Summary(summary="your summary"))
    ]),
    fallback=Summary(summary="error")
)

print(enrich_prompt.as_prompt())

Persona:
	- Name: Andy
	- Description: The best editor who can summarize anything.

Instructions:
	- summarize the context in concise manner.
	- do not make up your summary, it must be based on the context only.
Cautions:
	- respond in specified JSON format as in examples.

IMPORTANT:
	- Output only in a specified JSON format.
	- Do not include any premable, postmable, introductory, conversation, explanation or additional information.

Understand the following JSON schema and output only in a specified JSON format as in examples:
{
    "properties": {
        "summary": {
            "description": "a short summary based on the provided context.",
            "title": "Summary",
            "type": "string"
        }
    },
    "required": [
        "summary"
    ],
    "title": "Summary",
    "type": "object"
}

Example 1: "Academic journal"
Context: 
This is the input context.

Output:
```json
{
    "summary": "your summary"
}
```


In [24]:
from broai.experiments.bro_agent import BroAgent

In [25]:
enrich_agent = BroAgent(
    prompt_generator=enrich_prompt,
    model=bedrock_model
)

In [26]:
%%time
from tqdm import tqdm
summaries = []
errors = []
for enum, context in enumerate(tqdm(new_contexts.contexts)):
    try:
        response = enrich_agent.run(InputContext(context=context.context))
        summaries.append(
            Context(id=context.id, context=response.summary, metadata=context.metadata.copy())
        )
    except Exception as e:
        errors.append((enum, e, context))
len(summaries), len(errors)

100%|██████████| 63/63 [01:05<00:00,  1.03s/it]

CPU times: user 2.6 s, sys: 34.2 ms, total: 2.63 s
Wall time: 1min 5s


(63, 0)

In [27]:
summaries[0]

Context(id='30c93386-73ce-4082-83cc-5b4d9bc16201', context='Researchers from Stanford University propose using large language models to assist in writing Wikipedia-like articles from scratch.', metadata={'section': '# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024', 'source': '.docs/test1/storm.md', 'type': 'document', 'sequence': 0}, type='document', created_at='2025-04-27 00:05:58.417874')

In [29]:
enrich_vs = DuckVectorStore(db_name=DB_NAME, table="enrich", embedding=baai_em)

/tmp/ipykernel_15313/1870813766.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  enrich_vs = DuckVectorStore(db_name=DB_NAME, table="enrich", embedding=baai_em)


In [30]:
# enrich_vs.add_contexts(contexts=summaries)

In [31]:
enrich_vs.read_all()

,id,context,metadata,type,embedding,created_at,updated_at
0,30c93386-73ce-4082-83cc-5b4d9bc16201,Researchers from Stanford University propose u...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.022033691, 0.005504608, -0.036621094, -0.0...",2025-04-27 00:05:58.417874,2025-04-27 00:05:58.417874
1,17b93796-f7c1-4631-87db-2c6f1b9b8777,"Researchers propose STORM, a writing system to...","{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[-0.01360321, 0.021011353, -0.009269714, -0.01...",2025-04-27 00:05:59.450328,2025-04-27 00:05:59.450328
2,40339971-d18d-4ca4-a43d-017167bf5665,The paper explores the challenges of using lar...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.017730713, 0.01134491, -0.018203735, -0.00...",2025-04-27 00:06:00.664500,2025-04-27 00:06:00.664500
3,27714503-d35d-4156-97ba-1258d347a2d9,The paper studies generating Wikipedia-like ar...,"{""section"":""2 FreshWiki"",""source"":"".docs/test1...",document,"[-0.03237915, 0.008705139, -0.039886475, 0.028...",2025-04-27 00:06:01.790498,2025-04-27 00:06:01.790498
4,1ec83a5a-25c1-40ff-9d0a-8ba84debea3d,The FreshWiki Dataset is created by selecting ...,"{""section"":""2.1 The FreshWiki Dataset"",""source...",document,"[-0.06719971, -0.03564453, -0.040863037, 0.037...",2025-04-27 00:06:02.791599,2025-04-27 00:06:02.791599
...,...,...,...,...,...,...,...
58,8991673c-5911-486a-b340-b796726e4195,"Red Light Fever is the band's first album, rel...","{""section"":""**### Red Light Fever**"",""source"":...",document,"[0.011779785, -0.0071258545, -0.06854248, 0.00...",2025-04-27 00:06:58.718467,2025-04-27 00:06:58.718467
59,e74c371f-98f5-4fd9-b292-eba665a3d110,Taylor Hawkins & The Coattail Riders released ...,"{""section"":""**## Get the Money**"",""source"":"".d...",document,"[0.0077323914, -0.005973816, -0.035095215, 0.0...",2025-04-27 00:06:59.686173,2025-04-27 00:06:59.686173
60,f43a9814-f436-42ed-a3fe-23b87695879c,Taylor Hawkins collaborated with various artis...,"{""section"":""**# Collaborations and Guest Appea...",document,"[0.003293991, 0.0023384094, -0.057769775, 0.01...",2025-04-27 00:07:00.648724,2025-04-27 00:07:00.648724
61,bbdf145a-79b2-4c2c-bd0b-9197b54e55c3,"Taylor Hawkins, the drummer of Foo Fighters, p...","{""section"":""**# Tragic Passing**"",""source"":"".d...",document,"[0.014877319, 0.0068855286, -0.018829346, 0.01...",2025-04-27 00:07:01.675526,2025-04-27 00:07:01.675526


In [32]:
query = "What is STORM?"
retrieved_contexts = enrich_vs.vector_search(search_query=query, context=True)
reranked_raw, scores = rr.run(query, retrieved_contexts)
for res in reranked_raw:
    print(res.context)
    print("="*20)

STORM is an algorithm that generates an outline and references for a given topic, using a maximum number of perspectives and conversation rounds.
STORM is a framework that automates the pre-writing stage by discovering perspectives, simulating conversations, and creating outlines for long-form articles.
STORM is a system that automates the pre-writing stage by researching a given topic, creating an outline, and extending it to a full-length article grounded on collected references.
STORM simulates a conversation between a Wikipedia writer and a topic expert to generate questions and answers, using trusted sources and a rule-based filter to ensure factual information.
STORM creates an outline for an article by generating a draft outline from a topic and refining it with simulated conversations.


# Create: combo_vs

In [33]:
combo_vs = DuckVectorStore(db_name=DB_NAME, table="combo", embedding=baai_em)

/tmp/ipykernel_15313/2592293609.py:1: UserWarning: [EXPERIMENT] You're using an experimental module, which is subject to change in future.: DuckVectorStore
  combo_vs = DuckVectorStore(db_name=DB_NAME, table="combo", embedding=baai_em)


In [34]:
combo_contexts = []
for n, s in zip(new_contexts.contexts, summaries):
    if n.id == s.id:
        combo_contexts.append(
            Context(id=n.id, context=f"{s.context}\n\n{n.context}", metadata=n.metadata.copy())
        )


In [36]:
len(new_contexts.contexts), len(summaries), len(combo_contexts)

(63, 63, 63)

In [37]:
# combo_vs.add_contexts(contexts=combo_contexts)

In [38]:
combo_vs.read_all()

,id,context,metadata,type,embedding,created_at,updated_at
0,30c93386-73ce-4082-83cc-5b4d9bc16201,Researchers from Stanford University propose u...,"{""section"":""# arXiv:2402.14207v2 [cs.CL] 8 Apr...",document,"[-0.04071045, 0.0022087097, -0.026824951, -0.0...",2025-04-27 00:13:07.066147,2025-04-27 00:13:07.066147
1,17b93796-f7c1-4631-87db-2c6f1b9b8777,"Researchers propose STORM, a writing system to...","{""section"":""Abstract"",""source"":"".docs/test1/st...",document,"[-0.0007429123, -0.014602661, -0.023757935, -0...",2025-04-27 00:13:07.066189,2025-04-27 00:13:07.066189
2,40339971-d18d-4ca4-a43d-017167bf5665,The paper explores the challenges of using lar...,"{""section"":""1 Introduction"",""source"":"".docs/te...",document,"[-0.032928467, -0.0020828247, -0.022827148, -0...",2025-04-27 00:13:07.066205,2025-04-27 00:13:07.066205
3,27714503-d35d-4156-97ba-1258d347a2d9,The paper studies generating Wikipedia-like ar...,"{""section"":""2 FreshWiki"",""source"":"".docs/test1...",document,"[-0.024627686, 0.0016231537, -0.043518066, 0.0...",2025-04-27 00:13:07.066218,2025-04-27 00:13:07.066218
4,1ec83a5a-25c1-40ff-9d0a-8ba84debea3d,The FreshWiki Dataset is created by selecting ...,"{""section"":""2.1 The FreshWiki Dataset"",""source...",document,"[-0.06329346, -0.034179688, -0.038330078, 0.03...",2025-04-27 00:13:07.066230,2025-04-27 00:13:07.066230
...,...,...,...,...,...,...,...
58,8991673c-5911-486a-b340-b796726e4195,"Red Light Fever is the band's first album, rel...","{""section"":""**### Red Light Fever**"",""source"":...",document,"[0.0066490173, -0.04135132, -0.06628418, 0.027...",2025-04-27 00:13:07.066851,2025-04-27 00:13:07.066851
59,e74c371f-98f5-4fd9-b292-eba665a3d110,Taylor Hawkins & The Coattail Riders released ...,"{""section"":""**## Get the Money**"",""source"":"".d...",document,"[-0.0041770935, -0.012557983, -0.048309326, 0....",2025-04-27 00:13:07.066863,2025-04-27 00:13:07.066863
60,f43a9814-f436-42ed-a3fe-23b87695879c,Taylor Hawkins collaborated with various artis...,"{""section"":""**# Collaborations and Guest Appea...",document,"[-0.013771057, -0.031173706, -0.055999756, 0.0...",2025-04-27 00:13:07.066874,2025-04-27 00:13:07.066874
61,bbdf145a-79b2-4c2c-bd0b-9197b54e55c3,"Taylor Hawkins, the drummer of Foo Fighters, p...","{""section"":""**# Tragic Passing**"",""source"":"".d...",document,"[0.008712769, -0.027130127, -0.03933716, -0.00...",2025-04-27 00:13:07.066885,2025-04-27 00:13:07.066885


In [39]:
query = "What is STORM?"
retrieved_contexts = combo_vs.vector_search(search_query=query, context=True)
reranked_raw, scores = rr.run(query, retrieved_contexts)
for res in reranked_raw:
    print(res.context)
    print("="*20)

STORM is a framework that automates the pre-writing stage by discovering perspectives, simulating conversations, and creating outlines for long-form articles.

<span id="page-13-2"></span>B Pseudo Code of STORM

In [§3,](#page-2-8) we introduce STORM, a framework that automates the pre-writing stage by discovering different perspectives, simulating information-seeking conversations, and creating a comprehensive outline. Algorithm [1](#page-16-2) displays the skeleton of STORM.

We implement STORM with zero-shot prompting using the DSPy framework [\(Khattab et al.,](#page-10-9) [2023\)](#page-10-9). Listing [1](#page-14-0) and [2](#page-15-0) show the prompts used in our implementation. We highlight that STORM offers a general framework designed to assist the creation of grounded, long-form articles, without depending extensively on prompt engineering for a single domain.

STORM is a system that automates the pre-writing stage by researching a given topic, creating an outline, and exten

# Create: jargon_vs

In [40]:
bedrock_model = BedrockOllamaChat(model_name='us.meta.llama3-3-70b-instruct-v1:0')
# bedrock_model = BedrockOllamaChat(model_name='us.meta.llama3-2-3b-instruct-v1:0')

from typing import List

class Jargon(BaseModel):
    jargon:str = Field(description="the extracted jargon, acronym, abbreviation, proper name from the context"),
    evidence:str = Field(description="the evidence of the extracted jargon, acronym, abbreviation, proper name from the context"),
    explanation:str = Field(description="A short explanation based on the evidence.")

class Jargons(BaseModel):
    jargons:List[Jargon] = Field(description="A list of extracted jargons")

jargon_prompt = PromptGenerator(
    persona=Persona(name="Jame", description="The best proof reader who can spot any jargon, acronym, abbreviation and proper name at ease."),
    instructions=Instructions(
        instructions=[
            "extract jargon, acronym, abbrevation, proper name and its evidence.",
            "create a short explanation of the extracted jargon, acronym, abbrevation, perper name based on its evidence.",
            "always avoid extracting the person name",
            "do not make up your explanation",
        ],
        cautions=[
            "Remember not everything is jargon, acronym, abbrevation or proper name, so do not try to extract everything, try only obvious or certain."
            "Do not extract jargon, acronym, abbreviation or proper name that does not contain evidence",
            "if you are not sure if it's a jargon, acronym, abbreviation, proper name, do not extract it",
            "respond in specified JSON format as in examples"
        ]
    ),
    structured_output=Jargons,
    examples=Examples(examples=[
        Example(
            setting="Academic Journal",
            input=InputContext(context="Andrew Ng said, The new era of LLM, large language model, comes faster than anyone expected. It works well with RAG, retrieveal augmented generation, and it allows agentic framework to ..."),
            output=Jargons(jargons=[
                Jargon(
                    jargon="LLM",
                    evidence="The new era of LLM, large language model",
                    explanation="LLM is a large language model"
                ),
                Jargon(
                    jargon="RAG",
                    evidence="It works well with RAG, retrieval augmented generation",
                    explanation="RAG is short for retrieval augmented generation"
                )
            ])
        )
    ]),
    fallback=Jargons(jargons=[Jargon(jargon="error", evidence="error", explanation="error")])
)

In [41]:
jargon_agent = BroAgent(
    prompt_generator=jargon_prompt,
    model=bedrock_model
)

In [44]:
from broai.utils import get_timestamp
from datetime import datetime
from pydantic import BaseModel, Field
from uuid import uuid4
class JargonRecord(BaseModel):
    id:str = Field(description="unique id generated by system", default_factory=lambda: str(uuid4()))
    jargon:str
    evidence:str
    explanation:str
    metadata:dict
    created_at: datetime = Field(description="Timestamp Asia/Bangkok", default_factory=get_timestamp)

In [45]:
%%time
from tqdm import tqdm
jargons_contexts = []
errors = []
for enum, context in enumerate(tqdm(new_contexts.contexts[:])):
    try:
        response = jargon_agent.run(InputContext(context=context.context))
        metadata = context.metadata.copy()
        for j in response.jargons:
            if j.evidence.lower() != "none":
                jr = JargonRecord(jargon=j.jargon, evidence=j.evidence, explanation=j.explanation, metadata=metadata.copy())
                jargons_contexts.append(jr)
    except Exception as e:
        print(enum, "="*20)
        errors.append(
            (enum, str(e), context)
        )

  0%|          | 0/63 [00:00<?, ?it/s]/home/ec2-user/SageMaker/research_ai/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/ec2-user/SageMaker/research_ai/.venv/lib/python3.11/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='the evidence of the extracted jargon, acronym, abbreviation, proper name from the context'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
 25%|██▌       | 16/63 [00:30<01:48,  2.31s/it]/home/ec2-user/SageMaker/research_ai/.venv/lib/python3.11

Both parse_structured_output and content_extractor failed:
1 validation error for Jargons
  Invalid JSON: expected `,` or `}` at line 16 column 68 [type=json_invalid, input_value='\n{\n    "jargons": [\n ...\n        }\n    ]\n}\n', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid
BroAgent.parse_structured_output() got multiple values for argument 'text'


100%|██████████| 63/63 [01:50<00:00,  1.75s/it]

CPU times: user 2.74 s, sys: 51.2 ms, total: 2.79 s
Wall time: 1min 50s


In [46]:
len(jargons_contexts), len(errors)

(137, 0)

In [47]:
jargons_contexts[0]

JargonRecord(id='8257667d-d84a-4dd3-b546-be4a4454756c', jargon='arXiv', evidence='# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024', explanation='arXiv is an online archive of electronic preprints', metadata={'section': '# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024', 'source': '.docs/test1/storm.md', 'type': 'document', 'sequence': 0}, created_at='2025-04-27 00:16:48.130036')

In [98]:
import duckdb
from broai.experiments.huggingface_embedding import BaseEmbeddingModel
from typing import Dict, List, Any
from broai.duckdb_management.utils import get_create_table_query, get_insert_query
from pydantic import BaseModel, Field
from uuid import uuid4
from datetime import datetime
import json

class JargonRecord(BaseModel):
    id:str = Field(description="unique id generated by system", default_factory=lambda: str(uuid4()))
    jargon:str
    evidence:str
    explanation:str
    metadata:dict
    created_at: datetime = Field(description="Timestamp Asia/Bangkok", default_factory=get_timestamp)
    
class JargonStore:
    def __init__(self, db_name:str, table:str, limit:int=5):
        self.db_name = db_name
        self.table = table
        self.limit = limit
        self.__schemas = {
            "id": "VARCHAR",
            "jargon": "VARCHAR",
            "evidence": "VARCHAR",
            "explanation": "VARCHAR",
            "metadata": "JSON",
            "created_at": "TIMESTAMP",
            "updated_at": "TIMESTAMP",
        }
        self.create_table()
    
    def sql(self, query, params:Dict[str, Any]=None):
        with duckdb.connect(self.db_name) as con:
            con.sql(query, params=params)
            
    def sql_df(self, query, params:Dict[str, Any]=None):
        with duckdb.connect(self.db_name) as con:
            df = con.sql(query, params=params).to_df()
        return df

    def sql_records(self, df):
        df = df.drop("created_at", axis=1).copy()
        return [JargonRecord(id=record["id"], jargon=record["jargon"], evidence=record["evidence"], explanation=record["explanation"], metadata=json.loads(record["metadata"]), created_at=record["updated_at"], ) for record in df.to_dict(orient="records")]
    
    def create_table(self,):
        query = get_create_table_query(table=self.table, schemas=self.__schemas)
        self.sql(query)

    def read_all(self, query:str=None):
        if query is None:
            query = f"SELECT * FROM {self.table};"
        return self.sql_df(query)
    
    def add_jargons(self, jargons:List[JargonRecord]):
        id_list = [j.id for j in jargons]
        jargon_list = [j.jargon for j in jargons]
        evidence_list = [j.evidence for j in jargons]
        explanation_list = [j.explanation for j in jargons]
        metadata_list = [j.metadata for j in jargons]
        created_list = [j.created_at for j in jargons]
        rows = list(zip(id_list, jargon_list, evidence_list, explanation_list, metadata_list, created_list, created_list))
        with duckdb.connect(self.db_name) as con:
            con.executemany(f"INSERT INTO {self.table} (id, jargon, evidence, explanation, metadata, created_at, updated_at) VALUES (?, ?, ?, ?, ?, ?, ?)", rows)
        self.create_fts_index()

    def create_fts_index(self):
        query = f"""
        INSTALL fts;
        LOAD fts;
        PRAGMA create_fts_index(
            '{self.table}', 'id', 'jargon', overwrite=1
        );
        """.strip()
        self.sql(query)

    def fulltext_search(self, search_query:str, limit:int=None, context=True):
        if limit is None:
            limit = self.limit
        query = f"""\
        SELECT *
        FROM (
            SELECT *, fts_main_{self.table}.match_bm25(
                id,
                '{search_query}',
                fields := 'jargon'
            ) AS score
            FROM {self.table}
        ) sq
        ORDER BY score DESC
        LIMIT {limit}
        ;
        """
        df = self.sql_df(query=query, params=None)
        df = df.loc[~df['score'].isna(),:].copy()
        
        if df.shape[0]==0:
            return None
        if context is False:
            return df
        return self.sql_records(df=df)

    def filter_metadata(self, section='Abstract'):
        query = f"""
        SELECT *
        FROM {self.table}
        WHERE metadata ->> 'section' = '{section}';
        """
        return self.sql_df(query)

In [99]:
jargon_vs = JargonStore(db_name=DB_NAME, table="jargon_vs")

In [52]:
# jargon_vs.add_jargons(jargons=jargons_contexts)

In [55]:
jargon_vs.filter_metadata(section="1 Introduction")

,id,jargon,evidence,explanation,metadata,created_at,updated_at
0,642dcdc2-ad61-41c8-b280-54c5da909110,LLMs,Large language models (LLMs) have demonstrated...,LLMs are large language models,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-27 00:16:51.439281,2025-04-27 00:16:51.439281
1,e5e11401-8f8c-405b-9e66-cf7388daded8,RAG,current strategies often involve retrieval-aug...,RAG is short for retrieval-augmented generation,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-27 00:16:51.439540,2025-04-27 00:16:51.439540
2,2d38dbc3-fcc8-4612-b575-603b01c96297,STORM,we propose the STORM paradigm for the Synthesi...,STORM is a paradigm for the Synthesis of Topic...,"{""section"":""1 Introduction"",""source"":"".docs/te...",2025-04-27 00:16:51.439563,2025-04-27 00:16:51.439563


In [57]:
jargon_vs.read_all()['evidence'].value_counts()

evidence
Sentence-BERT: Sentence embeddings using Siamese BERT networks                                                                              3
their music spans genres including Hard Rock, Art Rock, and Alternative Rock                                                                3
# arXiv:2402.14207v2 [cs.CL] 8 Apr 2024                                                                                                     2
For the 1-7 scale rating results on each criterion, we calculate the Krippendorff's Alpha to measure the inter annotator agreement (IAA)    2
We propose STORM, an LLM-based writing system                                                                                               2
                                                                                                                                           ..
Nancy Wilson of Heart                                                                                                                      

In [62]:
jargon_vs.fulltext_search(search_query="What does FreshWiki stand for?").to_dict(orient="records")

[{'id': '2ed53744-5b50-4f41-a8b5-00c76211e186',
  'jargon': 'FreshWiki',
  'evidence': 'we randomly select 100 samples from the FreshWiki dataset',
  'explanation': 'FreshWiki is a dataset',
  'metadata': '{"section":"4 Experiments","source":".docs/test1/storm.md","type":"document","sequence":11}',
  'created_at': Timestamp('2025-04-27 00:17:06.814737'),
  'updated_at': Timestamp('2025-04-27 00:17:06.814737'),
  'score': 2.0913218534149}]

In [100]:
jargon_vs.fulltext_search(search_query="What does FreshWiki stand for?", context=True)

[JargonRecord(id='2ed53744-5b50-4f41-a8b5-00c76211e186', jargon='FreshWiki', evidence='we randomly select 100 samples from the FreshWiki dataset', explanation='FreshWiki is a dataset', metadata={'section': '4 Experiments', 'source': '.docs/test1/storm.md', 'type': 'document', 'sequence': 11}, created_at=Timestamp('2025-04-27 00:17:06.814737'))]

In [64]:
jargon_vs.read_all()['jargon'].value_counts().head(20)

jargon
STORM            21
LLM              12
LLMs              7
RAG               5
API               4
ACL               4
NLP               3
R                 3
AI                3
Foo Fighters      3
Likert scale      2
dspy              2
BERT              2
GPT-3.5           2
EMNLP             2
Sentence-BERT     2
DSPy              2
Queen             2
oRAG              2
FLAIR             1
Name: count, dtype: int64

In [66]:
jargon_vs.fulltext_search(search_query="oRAG")["evidence"].tolist()

['Outline-driven RAG (oRAG), which is identical to *RAG* in outline creation, but further searches additional information with section titles to generate the article section by section',
 'We conduct paired t-test and report the p-value. who have made at least 500 edits on Wikipedia and have more than 1 year of experience. We randomly sample 20 topics from our dataset and evaluate the articles generated by our method and *oRAG*, the best baseline according to the automatic evaluation.']

In [75]:
jargon_vs.fulltext_search(search_query="STORM")["explanation"].tolist()

['STORM is a system that automates the pre-writing stage',
 'STORM is a paradigm for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking',
 'STORM is a system that simulates conversations',
 'STORM is a system or method for discovering perspectives and controlling question asking',
 'STORM is a writing system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking']

In [74]:
jargon_vs.fulltext_search(search_query="oRAG")["explanation"].tolist()

['oRAG is an outline-driven retrieval-augmented generation',
 'oRAG is a baseline method or system for comparison']

In [73]:
jargon_vs.fulltext_search(search_query="LLM").to_dict(orient="records")

[{'id': '870847b9-632e-4448-856a-aead103ea739',
  'jargon': 'LLM',
  'evidence': "The final outline is curated based on the LLM's intrinsic knowledge",
  'explanation': 'LLM is a large language model',
  'metadata': '{"section":"3 Method","source":".docs/test1/storm.md","type":"document","sequence":6}',
  'created_at': Timestamp('2025-04-27 00:16:57.893432'),
  'updated_at': Timestamp('2025-04-27 00:16:57.893432'),
  'score': 0.9050358179756888},
 {'id': '642dcdc2-ad61-41c8-b280-54c5da909110',
  'jargon': 'LLMs',
  'evidence': 'Large language models (LLMs) have demonstrated impressive writing capabilities',
  'explanation': 'LLMs are large language models',
  'metadata': '{"section":"1 Introduction","source":".docs/test1/storm.md","type":"document","sequence":2}',
  'created_at': Timestamp('2025-04-27 00:16:51.439281'),
  'updated_at': Timestamp('2025-04-27 00:16:51.439281'),
  'score': 0.9050358179756888},
 {'id': '15eedf0f-50c1-49bb-9576-9c126d1af02c',
  'jargon': 'LLM',
  'evidence'

In [76]:
jargon_vs.fulltext_search(search_query="STORM").to_dict(orient="records")

[{'id': '71720532-85dd-45f3-8e2a-63eda0fe1937',
  'jargon': 'STORM',
  'evidence': 'We present STORM to automate the pre-writing stage',
  'explanation': 'STORM is a system that automates the pre-writing stage',
  'metadata': '{"section":"3 Method","source":".docs/test1/storm.md","type":"document","sequence":6}',
  'created_at': Timestamp('2025-04-27 00:16:57.892940'),
  'updated_at': Timestamp('2025-04-27 00:16:57.892940'),
  'score': 0.859878141615185},
 {'id': '2d38dbc3-fcc8-4612-b575-603b01c96297',
  'jargon': 'STORM',
  'evidence': 'we propose the STORM paradigm for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking',
  'explanation': 'STORM is a paradigm for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking',
  'metadata': '{"section":"1 Introduction","source":".docs/test1/storm.md","type":"document","sequence":2}',
  'created_at': Timestamp('2025-04-27 00:16:51.439563'),
  'updated_at': Timestamp('2025-04

# Test

In [78]:
from broai.prompt_management.core import PromptGenerator
from broai.prompt_management.interface import Persona, Instructions, Examples, Example
from pydantic import BaseModel, Field
from typing import List

## JargonDetector

In [90]:
bedrock_model = BedrockOllamaChat(model_name='us.meta.llama3-2-3b-instruct-v1:0')

class InputMessage(BaseModel):
    message:str = Field(desctiption="a user's input message")

class PotentialJargon(BaseModel):
    jargon:str = Field(description="a potential jargon, acronym, abbreviation, or proper name")
    confidence:float = Field(description="a score ranging from 0 to 1. 0 means you're not confident whereas 1 is you're really confident")

class PotentialJargons(BaseModel):
    jargons:List[PotentialJargon] = Field(description="a list of potential jargon")

jargon_detector_prompt = PromptGenerator(
    persona=Persona(name="Smith", description="You are the best peer reviewer who knows all jargons, acronyms, abbreviations and proper names and you can spot them all easily."),
    instructions=Instructions(
        instructions=[
            "read content carefully",
            "extract only jargon, acronym, abbrevation, or proper name",
            "in each extraction, give a confidence score ranging from 0 to 1. 0 means you're not confident whereas 1 is you're really confident"
            "always avoid extracting the person name",
        ],
        cautions=[
            "Remember not everything is jargon, acronym, abbrevation or proper name, so do not try to extract everything, try only obvious or certain."
            "if you are not sure if it's a jargon, acronym, abbreviation, proper name, do not extract it",
            "respond in specified JSON format as in examples"
        ]
    ),
    structured_output=PotentialJargons,
    examples=Examples(examples=[
        Example(
            setting="Academic Journal",
            input=InputMessage(message="Do you know what does LLM stand for?"),
            output=PotentialJargons(jargons=[
                PotentialJargon(
                    jargon="LLM",
                    confidence=.9
                )
            ])
        ),
        Example(
            setting="Blog Post",
            input=InputMessage(message="In the age of AI, LLM and RAG are the most implemented systems across all business sectors."),
            output=PotentialJargons(jargons=[
                PotentialJargon(
                    jargon="LLM",
                    confidence=1
                ),
                PotentialJargon(
                    jargon="RAG",
                    confidence=.9
                ),
            ])
        ),
    ]),
    fallback=PotentialJargons(jargons=[PotentialJargon(jargon="error", confidence=0)]),
)
print(jargon_detector_prompt.as_prompt())

Persona:
	- Name: Smith
	- Description: You are the best peer reviewer who knows all jargons, acronyms, abbreviations and proper names and you can spot them all easily.

Instructions:
	- read content carefully
	- extract only jargon, acronym, abbrevation, or proper name
	- in each extraction, give a confidence score ranging from 0 to 1. 0 means you're not confident whereas 1 is you're really confidentalways avoid extracting the person name
Cautions:
	- Remember not everything is jargon, acronym, abbrevation or proper name, so do not try to extract everything, try only obvious or certain.if you are not sure if it's a jargon, acronym, abbreviation, proper name, do not extract it
	- respond in specified JSON format as in examples

IMPORTANT:
	- Output only in a specified JSON format.
	- Do not include any premable, postmable, introductory, conversation, explanation or additional information.

Understand the following JSON schema and output only in a specified JSON format as in examples:
{

In [91]:
jargon_detector_agent = BroAgent(
    prompt_generator=jargon_detector_prompt,
    model=bedrock_model
)

In [106]:
input_msg = InputMessage(message="What does STORM use FreshWiki as a dataset?")
response = jargon_detector_agent.run(request=input_msg)
response

PotentialJargons(jargons=[PotentialJargon(jargon='STORM', confidence=0.9), PotentialJargon(jargon='FreshWiki', confidence=0.9)])

In [111]:
input_msg.message

'What does STORM use FreshWiki as a dataset?'

In [108]:
search_list = []
for res in response.jargons:
    if res.confidence > 0.5:
        search_list.append(res.jargon)
search_list

['STORM', 'FreshWiki']

In [109]:
retreived_jargons = []
for sq in search_list:
    retreived_jargons.extend(jargon_vs.fulltext_search(search_query=sq, context=True))
len(retreived_jargons)

6

In [110]:
for enum, j in enumerate(retreived_jargons):
    print(enum)
    print(j.jargon)
    print(j.evidence)
    print(j.explanation)
    print("="*20)

0
STORM
We present STORM to automate the pre-writing stage
STORM is a system that automates the pre-writing stage
1
STORM
we propose the STORM paradigm for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking
STORM is a paradigm for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking
2
STORM
STORM simulates a conversation between a Wikipedia writer and a topic expert
STORM is a system that simulates conversations
3
STORM
STORM discovers different perspectives by surveying existing articles from similar topics
STORM is a system or method for discovering perspectives and controlling question asking
4
STORM
We propose STORM, a writing system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking
STORM is a writing system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking
5
FreshWiki
we randomly select 100 samples from the FreshWiki dataset
Fr

## JargonEditor

In [175]:
bedrock_model = BedrockOllamaChat(model_name='us.meta.llama3-3-70b-instruct-v1:0') # most accurate
# bedrock_model = BedrockOllamaChat(model_name='us.meta.llama3-2-11b-instruct-v1:0') # accurate in meaning
# bedrock_model = BedrockOllamaChat(model_name='us.meta.llama3-2-3b-instruct-v1:0') # error from time to time

class InputEditMessage(BaseModel):
    knowledge:str = Field(description="knowledge of jargons, acronyms, abbreviations and proper names")
    message:str = Field(desctiption="a user's input message")

class EditedMessage(BaseModel):
    edited_message:str = Field(description="an edited message based on knowledge and message")

jargon_editor_prompt = PromptGenerator(
    persona=Persona(name="Jim", description="You're the best editor who can use your knowledge to edit any message for better clarification."),
    instructions=Instructions(
        instructions=[
            "read knowledge and message carefully",
            "use knowledge to edit the message for better clarification",
        ],
        cautions=[
            "respond in specified JSON format as in examples"
        ]
    ),
    structured_output=EditedMessage,
    examples=Examples(examples=[
        Example(
            setting="Academic Journal",
            input=InputEditMessage(
                knowledge="1: LLM\n Evidence: LLM (Large Language Model) is the advance topic of most research.\nExplanation: LLM is Large Language Model\n\n2: RAG\n Evidence: RAG, retrieval-aumented generation, allows chatbot to connect with the internal knowledge\nExplanation: RAG stands for retrieval-aumented generation",
                message="What are the reasons for using LLM and RAG together?"
            ),
            output=EditedMessage(edited_message="What are the reasons for using LLM, Large Language Model, and RAG, Retrieval-Augmented Generation, together?")
        ),
    ]),
    fallback=EditedMessage(edited_message="error"),
)
print(jargon_detector_prompt.as_prompt())

Persona:
	- Name: Smith
	- Description: You are the best peer reviewer who knows all jargons, acronyms, abbreviations and proper names and you can spot them all easily.

Instructions:
	- read content carefully
	- extract only jargon, acronym, abbrevation, or proper name
	- in each extraction, give a confidence score ranging from 0 to 1. 0 means you're not confident whereas 1 is you're really confidentalways avoid extracting the person name
Cautions:
	- Remember not everything is jargon, acronym, abbrevation or proper name, so do not try to extract everything, try only obvious or certain.if you are not sure if it's a jargon, acronym, abbreviation, proper name, do not extract it
	- respond in specified JSON format as in examples

IMPORTANT:
	- Output only in a specified JSON format.
	- Do not include any premable, postmable, introductory, conversation, explanation or additional information.

Understand the following JSON schema and output only in a specified JSON format as in examples:
{

In [176]:
jargon_editor_agent = BroAgent(
    prompt_generator=jargon_editor_prompt,
    model=bedrock_model
)

In [177]:
knowledge = "\n".join([f"{enum+1}: {j.jargon}\n{j.evidence}\n{j.explanation}" for enum, j in enumerate(retreived_jargons)])
print(knowledge)

1: STORM
We present STORM to automate the pre-writing stage
STORM is a system that automates the pre-writing stage
2: STORM
we propose the STORM paradigm for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking
STORM is a paradigm for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking
3: STORM
STORM simulates a conversation between a Wikipedia writer and a topic expert
STORM is a system that simulates conversations
4: STORM
STORM discovers different perspectives by surveying existing articles from similar topics
STORM is a system or method for discovering perspectives and controlling question asking
5: STORM
We propose STORM, a writing system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking
STORM is a writing system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking
6: FreshWiki
we randomly select 100 samples from the FreshWiki data

In [178]:
response = jargon_editor_agent.run(request=InputEditMessage(knowledge=knowledge, message="How does STORM perform?"))
response

EditedMessage(edited_message='How does STORM, a system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking, perform?')

In [179]:
response = jargon_editor_agent.run(request=InputEditMessage(knowledge=knowledge, message="How does FreshWiki contribute to the study?"))
response

EditedMessage(edited_message='How does the FreshWiki dataset contribute to the study?')

In [180]:
input_msg.message

'What does STORM use FreshWiki as a dataset?'

In [181]:
response = jargon_editor_agent.run(request=InputEditMessage(knowledge=knowledge, message=input_msg.message))
response

EditedMessage(edited_message='What does STORM, a system for the Synthesis of Topic Outlines through Retrieval and Multi-perspective Question Asking, use FreshWiki, a dataset, as a dataset for?')